In [1]:
import os
import pandas as pd
from pandas import DataFrame

def merge_data(station_paths:list[str],info:str)->DataFrame:
    '''
    傳出所有的資料
    '''
    info_df = pd.read_csv(info)
    merge_datas = []
    for sta_path in station_paths:
        station_df = pd.read_csv(sta_path)
        merge_df = pd.merge(station_df, info_df,how='left',left_on='staCode',right_on='stationCode')
        merge_datas.append(merge_df)
    return pd.concat(merge_datas)


current_folder = os.getcwd()
stations_folder = os.path.join(current_folder,'stations')
csv_abs_files = []
for filename in os.listdir(stations_folder):
    if '每日各站' in filename:
        csv_abs_files.append(os.path.join(stations_folder,filename))

info_path = os.path.join(stations_folder,'台鐵車站資訊.csv')


df = merge_data(csv_abs_files,info_path)
df1 = df[['trnOpDate','gateInComingCnt','gateOutGoingCnt','name','stationAddrTw','haveBike']]

In [ ]:
df1.columns = ['日期','進站人數','出站人數','站名','地址','youbike']
df1

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 0 to 86551
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   日期       406761 non-null  int64 
 1   進站人數     406761 non-null  int64 
 2   出站人數     406761 non-null  int64 
 3   站名       406761 non-null  object
 4   地址       406761 non-null  object
 5   youbike  406761 non-null  object
dtypes: int64(3), object(3)
memory usage: 21.7+ MB


In [4]:
df1['日期'] = pd.to_datetime(df1['日期'].astype(str))

/tmp/ipykernel_14817/2470081304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['日期'] = pd.to_datetime(df1['日期'].astype(str))


In [5]:
df1['youbike'] = df1['youbike'].map({'Y':1,'N':0})

/tmp/ipykernel_14817/1675081192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['youbike'] = df1['youbike'].map({'Y':1,'N':0})


In [6]:
df2 = df1.sort_values(by='日期')
df3 = df2.set_index('日期')

In [ ]:
df3

In [ ]:
def first_third(val:str)->str:
    return val[:3]

df3['地址'].map(first_third)

In [9]:
df3['縣市'] = df3['地址'].map(lambda v:v[:3])

In [10]:
df3["年度"] = df3.index.strftime('%Y')
df3['年月'] = df3.index.strftime('%Y-%m')

In [ ]:
df4 = df3.reindex(columns=['縣市','年度','年月','站名','進站人數','出站人數','youbike'])
df4

In [12]:
df4.to_csv('全台火車進出站資訊.csv')